### After understanding the Intution part about Decision Tree Let's understand the implementation part of Decision Tree

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Import Required Libraries**

In [44]:
import numpy as np
import pandas as pd
import seaborn as sns

Load the dataset `'health_lifestyle_dataset.csv'` using pandas

In [45]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/sherians/datasets/health_lifestyle_dataset.csv")

**View top 5 rows of the dataset.**

In [46]:
df.head()

,id,age,gender,bmi,daily_steps,sleep_hours,water_intake_l,calories_consumed,smoker,alcohol,resting_hr,systolic_bp,diastolic_bp,cholesterol,family_history,disease_risk
0,1,56,Male,20.5,4198,3.9,3.4,1602,0,0,97,161,111,240,0,0
1,2,69,Female,33.3,14359,9.0,4.7,2346,0,1,68,116,65,207,0,0
2,3,46,Male,31.6,1817,6.6,4.2,1643,0,1,90,123,99,296,0,0
3,4,32,Female,38.2,15772,3.6,2.0,2460,0,0,71,165,95,175,0,0
4,5,60,Female,33.6,6037,3.8,4.0,3756,0,1,98,139,61,294,0,0


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 100000 non-null  int64  
 1   age                100000 non-null  int64  
 2   gender             100000 non-null  object 
 3   bmi                100000 non-null  float64
 4   daily_steps        100000 non-null  int64  
 5   sleep_hours        100000 non-null  float64
 6   water_intake_l     100000 non-null  float64
 7   calories_consumed  100000 non-null  int64  
 8   smoker             100000 non-null  int64  
 9   alcohol            100000 non-null  int64  
 10  resting_hr         100000 non-null  int64  
 11  systolic_bp        100000 non-null  int64  
 12  diastolic_bp       100000 non-null  int64  
 13  cholesterol        100000 non-null  int64  
 14  family_history     100000 non-null  int64  
 15  disease_risk       100000 non-null  int64  
dtypes: 

**Check for total null values present in our dataset**

In [48]:
df.isnull().sum()

,0
id,0
age,0
gender,0
bmi,0
daily_steps,0
sleep_hours,0
water_intake_l,0
calories_consumed,0
smoker,0
alcohol,0


**Now we have a column named as `gender` and this column consist of 'male' and 'female' which is of string datatype so we need to encode this in `0` and `1` for this we will use `LabelEncoder` from `sklearn.preprocessing`**

In [49]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['gender'] = le.fit_transform(df['gender'])

Now, let’s divide our dataset into two parts:

**X → Independent variables (features) – the inputs used to make predictions.**

**y → Dependent variable (target) – the output we want to predict.**

In [50]:
X = df.drop('disease_risk', axis=1)
y = df['disease_risk']

**Once we have separated the dataset into X (features) and y (target), the next step is to split the data into training and testing sets, and then use the training set to fit (train) our model.**

In [51]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

**We initialize a Decision Tree Classifier with entropy as the criterion, a maximum depth of 3, and default values for min_samples_leaf and min_samples_split, while fixing random_state=42 for reproducibility.**

In [52]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()

**As we have successfully created our model now it's time for training X_train and y_train**

In [53]:
dt.fit(X_train,y_train)

DecisionTreeClassifier()

**After training it's time for making prediciton on our decision tree model**

In [54]:
y_pred = dt.predict(X_test)

**After making predicition we have to check how good our model is performing after making prediction**

In [55]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.612030303030303

### Use RandomizedSearchCV to test a fixed number of random hyperparameter combinations for a Decision Tree with cross-validation, selecting the best set based on accuracy.
### It then trains the optimal model, makes predictions on the test data, and prints the best parameters along with the model’s accuracy.

In [56]:
from sklearn.model_selection import RandomizedSearchCV
param_grid = {
    'max_depth' :[3,5,7,9,None],
    'min_samples_split' : [2,5,10],
    'min_samples_leaf' : [1,2,4],
    'criterion' : ['gini','entropy']
}

In [40]:
rand_search = RandomizedSearchCV(
    estimator=DecisionTreeClassifier(random_state=42),
    param_distributions=param_grid,
    n_iter=20,
    cv = 5,
    scoring='accuracy',
    random_state=42,
    n_jobs=-1
)
rand_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 7, 9, None],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10]},
                   random_state=42, scoring='accuracy')

In [41]:
print("best parameters",rand_search.best_params_)
best_dt = rand_search.best_estimator_


best parameters {'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 3, 'criterion': 'entropy'}


In [42]:
y_pred = best_dt.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7507272727272727
